### Елизавета Клыкова, БКЛ181
# Домашнее задание № 2. Сравнение морфологических тэггеров

1. Написать русский и английский тексты (100+ слов), содержащие какие-либо трудные или неоднозначные для POS-теггинга моменты. Разметить их (только часть речи!). Объяснить, какие моменты являются трудными/неоднозначными и почему. (2 балла)
2. Проанализировать тексты с помощью трех POS-теггеров для каждого языка: pymorphy2, mystem, Natasha для русского; SpaCy, Flair, NLTK для английского. (4 балла)
3. Оценить accurary для каждого теггера. Для этого сначала нужно свести разметку к единому стандарту с помощью отдельной функции, а затем с помощью другой функции сравнить полученные результаты с размеченным вручную эталоном. (2 балла)
4. Выбрать лучший теггер для русского языка и написать функцию, повышающую качество работы программы из д/з 1. Выделить 3 вида синтаксических групп (не + какая-либо часть речи или N/NP + наречие и т.д.), использование которых могло бы улучшить работу программы. Написать функцию, выделяющую такие группы в тексте, используя что-либо из списка: chunking и regexp grammar, Natasha syntax parser, код с последнего занятия по SpaCy. (2 балла за функцию + 1 балл за пояснение, почему нужны именно такие группы).
5. Бонус. Встроить эту функцию в программу для д/з 1 и сравнить качество работы программы с ней и без нее (2 балла).

**Примечание для проверяющего:**
1) Файлы rus_text и eng_text содержат исходный текст. Для удобства он так же выводится в программе.
2) Файлы rus_tokens и eng_tokens содержат словоформы, записанные каждая с новой строки. Эти файлы создаются программой и дальше никак не используются.
3) Файлы rus_tokens2 и eng_tokens2 - копии файлов из предыдущего файла, в которые фручную внесена разметка частеречная разметка. (Я создала копии, т.к. люблю перезапустить программу и потерять все изменения в созданных файлах))))
4) Файлы rus_ann и eng_ann содержат полные таблицы с разметкой всеми нужными парсерами. Туда также входят леммы (для каждого парсера свои, для ручной разметки лемм нет). Можно заглянуть в эти файлы, чтобы увидеть, что выделенные мною проблемные места действительно оказались проблемными.

## Пункт 1: подготовка текстов

Перед началом работы импортируем все необходимые модули.

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import collections
import navec
import nltk
import pandas as pd
import re
import requests
import spacy
import string
from collections import Counter
from collections import OrderedDict
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter
from ipymarkup import show_span_ascii_markup as show_markup
from navec import Navec
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from pymystem3 import Mystem
from sklearn.metrics import accuracy_score
from slovnet import NER
from string import punctuation
from tqdm.auto import tqdm
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)

Русский и английский тексты были записаны в файлы rus_text.txt и eng_text.txt соответственно. Для удобства разметки считаем тексты, очистим их он пунктуации и разделим на слова, а затем запишем в новый файл (каждое слово на новой строке, после слова табуляция - нужно, чтобы потом этот файл можно было вручную доразметить и загрузить в pandas dataframe).

В качестве основы для разметки была взята разметка, принятая в НКРЯ (https://ruscorpora.ru/new/corpora-morph.html).

Поскольку разным парсерам может потребоваться разный формат данных, считаем тексты из файлов и запишем каждый в три разных переменных: text (сырой текст), word_list (список словоформ, очищенных от всей пунктуации, кроме дефиса и апострофа) и token_list (список всех токенов текста, включая пунктуацию).

In [3]:
def tokenize_text(filename, new_file):
    with open(filename, encoding='utf-8') as f:
        text = f.read()
    token_list = word_tokenize(text)
    punct = list(re.sub("[-']", '', string.punctuation))
    other_punct = ['``', '\"\"', '...', '--', '–', '—',
                   '«', '»', '“', '”', '’', '***', '…']
    all_punct = punct + other_punct
    rx = '[' + re.escape(''.join(all_punct)) + ']'
    word_list = word_tokenize((re.sub(rx, ' ', text)))
    token_text = '\t\n'.join(word_list)
    with open(new_file, 'w', encoding='utf-8') as f1:
        f1.write(token_text)
    return text, word_list, token_list

In [4]:
rus_text, rus_words, rus_tokens = tokenize_text(
    'rus_text.txt', 'rus_tokens.txt')
eng_text, eng_words, eng_tokens = tokenize_text(
    'eng_text.txt', 'eng_tokens.txt')

Посмотрим на тексты с точки зрения неоднозначных для автоматической разметки моментов.

In [5]:
rus_text

'- Я на мели, - сходу предупреждаю я, снимая пальто и проходя в комнату. Навстречу выходит Алланова собака, ньюфка, которая за полгода догнала и перегнала меня в весе. Теперь на ней можно ездить верхом.\n- Никогда такого не было и вот опять, - говорит Аллан и закатывает глаза. Все такая же язва. На душе у меня вдруг становится спокойно.\nСолнце уже село, а единственная в комнате люстра, в свое время купленная в секонде, приказала долго жить еще до прошлой вечеринки. Конечно, времени починить ее за полгода у Аллана не нашлось. Я свечу ему фонариком на телефоне, пока он достает из шкафа старую оплавившуюся свечу и спички.\n- Скоро всемирный день лени. Я подарю тебе лампу как главному лентяю.\n- А когда там всемирный день шила в одном месте? Надо не забыть тебя поздравить.\nМы беззлобно переругиваемся еще с четверть часа, когда звонит Ирэн и просит помочь ей донести сумки с продуктами от метро. Аллан говорит, что выйдет ей навстречу, и добавляет:\n- Надеюсь, Салли не задушит мою собаку, п

Неоднозначными моментами в тексте являются, например, частичные омонимы (мели, верхом, село, пища и т.д.) и неизменяемые слова, которые могут относиться к разным частям речи (навстречу как предлог и наречие, всё как наречие). Кроме того, присутствуют редкие нерусские имена (Салли) и лексемы, которые не встречаются в литературном русском: ньюфка (собака породы ньюфаундленд), секонд (магазин секонд-хенд).

In [6]:
eng_text

"Finally, drinks are served, and Simon drinks his beer as though it's the most delicious thing he's had in his entire life. We talk about the most random things, from the new lead he has in his case about lead poisoning to that god-awful seminar he is going to lead in two weeks. We end up discussing the works of a modern Nauruan artist whose name I don't even hope to remember, and after another round of drinks (tequila this time) I tell Simon about a guy that works in the office next door and has the most round face imagineable. Around midnight there is no more room for anything liquid in our stomachs and no cash in our wallets, so we decide to head out. Simon's car is parked around the corner and at this point he wants nothing more than to get in it and sleep it off for the next two weeks, but I point towards a cab waiting on the other side of the road, make Simon turn around and practically drag him across the street."

В английском тексте тоже много неоднозначных моментов, которые могут быть размечены неверно, если парсер не учитывает контекст. В числе таких моментов омонимичные слова (drinks, lead - 3 варианта!, round), слова типа around (предлог и наречие) и редкие слова (Nauruan - прилагательное, образованное от имени собственного Nauru - названия карликового государства).

Создав копии файлов, внесем в них ручную разметку (я создаю именно копии, чтобы не потерять разметку при возможном перезапуске программы целиком). Добавим названия столбцов в первую строку. Затем прочитаем файлы и преобразуем их в pandas dataframe.

In [7]:
rus_ann = pd.read_csv('rus_tokens2.txt', sep='\t')
eng_ann = pd.read_csv('eng_tokens2.txt', sep='\t')

## Пункт 2: использование парсеров
### Пункт 2.1. Русский текст
#### Pymorphy2
Начнем с разметки русского текста.

In [8]:
morph = MorphAnalyzer()

In [9]:
pm_parsed = [morph.parse(w)[0] for w in rus_words]
pm_lemmas = [lem.normal_form for lem in pm_parsed]
pm_POS = [str(lem.tag.POS) for lem in pm_parsed]

In [10]:
pm_POS = [str(lem.tag.POS) for lem in pm_parsed]

In [11]:
len(pm_POS)

200

Прежде чем записывать результат разбора в датафрейм, убедимся, что число словоформ совпадает:

In [12]:
len(rus_words)

200

In [13]:
rus_ann['pm2_lemmas'] = pm_lemmas
rus_ann['pm2_POS'] = pm_POS
rus_ann

,word,POS,pm2_lemmas,pm2_POS
0,Я,SPRON,я,NPRO
1,на,PREP,на,PREP
2,мели,NOUN,мелить,VERB
3,сходу,ADV,сходу,ADVB
4,предупреждаю,VERB,предупреждать,VERB
...,...,...,...,...
195,ли,PART,ли,PRCL
196,не,PART,не,PRCL
197,пища,VERB,пища,NOUN
198,от,PREP,от,PREP


#### Mystem
На вход подаем "сырой" текст.

In [14]:
m = Mystem()
ms_parsed = m.analyze(rus_text)

In [15]:
ms_lemmas = []
ms_POS = []
for token in ms_parsed:
    if 'analysis' in token:
        lemma = token['analysis'][0]['lex']
        pos = token['analysis'][0]['gr'].split('=')[0].split(',')[0]
        ms_lemmas.append(lemma)
        ms_POS.append(pos)

In [16]:
len(ms_POS)

200

In [17]:
rus_ann['ms_lemmas'] = ms_lemmas
rus_ann['ms_POS'] = ms_POS
rus_ann

,word,POS,pm2_lemmas,pm2_POS,ms_lemmas,ms_POS
0,Я,SPRON,я,NPRO,я,SPRO
1,на,PREP,на,PREP,на,PR
2,мели,NOUN,мелить,VERB,мель,S
3,сходу,ADV,сходу,ADVB,сходу,ADV
4,предупреждаю,VERB,предупреждать,VERB,предупреждать,V
...,...,...,...,...,...,...
195,ли,PART,ли,PRCL,ли,PART
196,не,PART,не,PRCL,не,PART
197,пища,VERB,пища,NOUN,пища,S
198,от,PREP,от,PREP,от,PR


#### Natasha

In [18]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
morph_vocab = MorphVocab()

In [19]:
doc = Doc(rus_text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [20]:
nt_parsed = [t for t in doc.tokens if t.pos != 'PUNCT']
nt_POS = [t.pos for t in nt_parsed]

In [21]:
len(nt_POS)

200

In [22]:
for token in nt_parsed:
    token.lemmatize(morph_vocab)
nt_lemmas = [t.lemma for t in nt_parsed]

In [23]:
len(nt_lemmas)

200

In [24]:
rus_ann['nt_lemmas'] = nt_lemmas
rus_ann['nt_POS'] = nt_POS
rus_ann

,word,POS,pm2_lemmas,pm2_POS,ms_lemmas,ms_POS,nt_lemmas,nt_POS
0,Я,SPRON,я,NPRO,я,SPRO,я,PRON
1,на,PREP,на,PREP,на,PR,на,ADP
2,мели,NOUN,мелить,VERB,мель,S,мель,NOUN
3,сходу,ADV,сходу,ADVB,сходу,ADV,сход,NOUN
4,предупреждаю,VERB,предупреждать,VERB,предупреждать,V,предупреждать,VERB
...,...,...,...,...,...,...,...,...
195,ли,PART,ли,PRCL,ли,PART,ли,PART
196,не,PART,не,PRCL,не,PART,не,PART
197,пища,VERB,пища,NOUN,пища,S,пища,NOUN
198,от,PREP,от,PREP,от,PR,от,ADP


### Пункт 2.2. Английский текст
#### SpaCy

In [25]:
nlp = spacy.load('en_core_web_sm')
doc2 = nlp(eng_text)

sp_lemmas = []
sp_POS = []
for sent in doc2.sents:
    lemmas = [t.lemma_ for t in sent if t.pos_ != 'PUNCT']
    pos = [t.pos_ for t in sent if t.pos_ != 'PUNCT']
    sp_lemmas.extend(lemmas)
    sp_POS.extend(pos)

In [26]:
len(sp_POS)

185

In [27]:
eng_ann['sp_lemmas'] = sp_lemmas
eng_ann['sp_POS'] = sp_POS
eng_ann

,word,POS,sp_lemmas,sp_POS
0,Finally,ADV,finally,ADV
1,drinks,NOUN,drink,NOUN
2,are,AUX,be,AUX
3,served,VERB,serve,VERB
4,and,CONJ,and,CCONJ
...,...,...,...,...
180,drag,VERB,drag,VERB
181,him,SPRON,-PRON-,PRON
182,across,PREP,across,ADP
183,the,DET,the,DET


#### Flair
Поскольку Flair не умеет лемматизировать словоформы, получим только части речи.

In [28]:
tagger = SequenceTagger.load('pos')
splitter = SegtokSentenceSplitter()

2020-10-18 17:25:34,669 loading file C:\Users\Елизавета Клыкова\.flair\models\en-pos-ontonotes-v0.5.pt


In [29]:
sentences = splitter.split(eng_text)

In [30]:
fl_tagged = []
for sent in sentences:
    tagger.predict(sent)
    fl_tagged.append(sent.to_tagged_string())
fl_tagged = ' '.join(fl_tagged)

In [31]:
fl_POS = re.findall('<[A-Z|$]+?>', fl_tagged)

In [32]:
len(fl_POS)

184

Здесь возникает проблема: в тексте есть слово "god-awful", которое SpaCy считает за два слова, а flair - за одно. На самом деле это более верный подход, но в ручной разметке это слово разбито на два, поэтому придется "испортить" разметку flair'а, добавив в нее отдельную часть речи для слова "god". Чтобы это сделать, получим номер этого слова, обратившись к списку лемм, полученному от SpaCy (работает, т.к. словоформа "god" встречается в тексте один раз):

In [33]:
idx = sp_lemmas.index('god')
idx

45

Теперь добавим в список fl_POS тег NN на место 45-го элемента, а остальные сдвинем:

In [34]:
new_fl_POS = fl_POS[0:45] + ['NN'] + fl_POS[45::]

In [35]:
len(new_fl_POS)

185

Очевидно, что этот метод далек от оптимального. Я решаю проблему именно так, поскольку нас интересует сравнение разных теггеров и совпадение количества словоформ является ключевым моментом. Кроме того, в нашем тексте такая ситуация встретилась только один раз, поэтому представляется возможным просто обратиться к слову по индексу и добавить нужный тег. В реальной задаче по разметке такой подход невозможен, а главное, не имеет смысла: flair в данном случае прав, и дорабатывать нужно не его разметку, а разметку тех парсеров, которые разбивают такие лексемы на две части.

In [36]:
eng_ann['fl_POS'] = [re.sub('[<|>]', '', pos) for pos in new_fl_POS]
eng_ann

,word,POS,sp_lemmas,sp_POS,fl_POS
0,Finally,ADV,finally,ADV,RB
1,drinks,NOUN,drink,NOUN,NNS
2,are,AUX,be,AUX,VBP
3,served,VERB,serve,VERB,VBN
4,and,CONJ,and,CCONJ,CC
...,...,...,...,...,...
180,drag,VERB,drag,VERB,VB
181,him,SPRON,-PRON-,PRON,PRP
182,across,PREP,across,ADP,IN
183,the,DET,the,DET,DT


#### NLTK
Поскольку NLTK не учитывает контекст, подадим на вход просто список словоформ - так будет быстрее.

In [37]:
nltk_tagged = nltk.pos_tag(eng_words)

In [38]:
wnl = WordNetLemmatizer()

nltk_POS = []
nltk_lemmas = []
for word in nltk_tagged:
    pos = nltk.tag.mapping.map_tag('en-ptb', 'universal', word[1])
    nltk_POS.append(pos)
    if pos == 'ADJ':
        lem_pos = 'a'
    elif pos == 'ADJ_SAT':
        lem_pos = 's'
    elif pos == 'ADV':
        lem_pos = 'r'
    elif pos == 'VERB':
        lem_pos = 'v'
    else:
        lem_pos = 'n'  # дефолтный тег
    lemma = wnl.lemmatize(word[0], pos=lem_pos)
    nltk_lemmas.append(lemma)

In [39]:
len(nltk_lemmas)

185

In [40]:
eng_ann['nltk_lemmas'] = nltk_lemmas
eng_ann['nltk_POS'] = nltk_POS
eng_ann

,word,POS,sp_lemmas,sp_POS,fl_POS,nltk_lemmas,nltk_POS
0,Finally,ADV,finally,ADV,RB,Finally,ADV
1,drinks,NOUN,drink,NOUN,NNS,drink,NOUN
2,are,AUX,be,AUX,VBP,be,VERB
3,served,VERB,serve,VERB,VBN,serve,VERB
4,and,CONJ,and,CCONJ,CC,and,CONJ
...,...,...,...,...,...,...,...
180,drag,VERB,drag,VERB,VB,drag,VERB
181,him,SPRON,-PRON-,PRON,PRP,him,PRON
182,across,PREP,across,ADP,IN,across,ADP
183,the,DET,the,DET,DT,the,DET


## Пункт 3: оценка accuracy
### Пункт 3.1. Стандартизация разметки

In [41]:
def unify_tags(tags, lang):
    new_tags = []
    nouns = ['NOUN', 'PROPN', 'S', 'NNS', 'NNP', 'NN']
    verbs = ['VERB', 'V', 'AUX', 'INFN', 'GRND',
             'VBP', 'VBZ', 'VBN', 'VBG', 'VB', 'PRTF']
    adverbs = ['ADV', 'ADVB', 'ADVPRO', 'PRED', 'RB', 'RBS']
    adjectives = ['ADJ', 'A', 'ADJS', 'ADJF', 'JJ', 'JJR']
    conjunctions = ['CONJ', 'SCONJ', 'CCONJ', 'CC']
    pronouns = ['PRON', 'SPRON', 'APRON', 'SPRO', 'APRO',
                'NPRO', 'PRP$', 'PRP', 'EX', 'RP', 'WP$']
    prepositions = ['PREP', 'PR', 'ADP']
    particles = ['PART', 'PRCL', 'POSS', 'POS', 'TO', 'PRT']
    numerals = ['NUM', 'CD']
    determiners = ['DET', 'DT', 'WDT']
    other = ['INTR', 'IN']  # IN у Flair - и предлог, и союз
    for t in tags:
        if t in nouns:
            new_tags.append('NOUN')
        elif t in verbs:
            new_tags.append('VERB')
        elif t in adverbs:
            new_tags.append('ADV')
        elif t in adjectives:
            new_tags.append('ADJ')
        elif t in conjunctions:
            new_tags.append('CONJ')
        elif t in pronouns:
            new_tags.append('PRON')
        elif t in prepositions:
            new_tags.append('PREP')
        elif t in particles:
            new_tags.append('PRCL')
        elif t in numerals:
            new_tags.append('NUM')
        elif t in determiners and lang == 'eng':
            new_tags.append('DET')
        elif t in other:
            new_tags.append('other')
        elif t == 'DET' and lang == 'rus':
            new_tags.append('PRON')
        else:
            new_tags.append('unknown')
    return new_tags

Применим получившуюся функцию к нужным столбцам датафреймов.

In [42]:
rus_ann['POS'] = unify_tags(list(rus_ann['POS']), 'rus')
rus_ann['pm2_POS'] = unify_tags(list(rus_ann['pm2_POS']), 'rus')
rus_ann['ms_POS'] = unify_tags(list(rus_ann['ms_POS']), 'rus')
rus_ann['nt_POS'] = unify_tags(list(rus_ann['nt_POS']), 'rus')
eng_ann['POS'] = unify_tags(list(eng_ann['POS']), 'eng')
eng_ann['sp_POS'] = unify_tags(list(eng_ann['sp_POS']), 'eng')
eng_ann['fl_POS'] = unify_tags(list(eng_ann['fl_POS']), 'eng')
eng_ann['nltk_POS'] = unify_tags(list(eng_ann['nltk_POS']), 'eng')

Сохраним датафреймы с аннотацией в отдельные файлы:

In [43]:
rus_ann.to_csv('rus_ann.tsv', sep='\t')
eng_ann.to_csv('eng_ann.tsv', sep='\t')

Теперь определим accuracy каждого парсера:

In [44]:
def test_pos_tagger(my_pos, tagger_pos, tagger_name):
    my_pos = list(my_pos)
    tagger_pos = list(tagger_pos)
    ignore = ['other', 'unknown']
    real_pos = []
    predicted_pos = []
    for i in range(len(my_pos)):
        if my_pos[i] not in ignore and tagger_pos[i] not in ignore:
            real_pos.append(my_pos[i])
            predicted_pos.append(tagger_pos[i])
    res = tagger_name + ' accuracy: ' + str(
        round(accuracy_score(predicted_pos, real_pos), 2))
    return res

In [45]:
test_pos_tagger(rus_ann['POS'], rus_ann['pm2_POS'], 'Pymorphy2')

'Pymorphy2 accuracy: 0.9'

In [46]:
test_pos_tagger(rus_ann['POS'], rus_ann['ms_POS'], 'Mystem')

'Mystem accuracy: 0.94'

In [47]:
test_pos_tagger(rus_ann['POS'], rus_ann['nt_POS'], 'Natasha')

'Natasha accuracy: 0.91'

In [48]:
test_pos_tagger(eng_ann['POS'], eng_ann['sp_POS'], 'SpaCy')

'SpaCy accuracy: 0.91'

In [49]:
test_pos_tagger(eng_ann['POS'], eng_ann['fl_POS'], 'Flair')

'Flair accuracy: 0.91'

In [50]:
test_pos_tagger(eng_ann['POS'], eng_ann['nltk_POS'], 'NLTK')

'NLTK accuracy: 0.89'

## Пункт 4: доработка программы из д/з 1
Вполне ожидаемо, Mystem показал наибольшую точность pos-tagging'a, поэтому в этой части использовать будем именно его.

Для начала определим, какие виды синтаксических групп нас интересуют.
1. Во-первых, это сочетание "не + глагол" (ему соответствуют биграммы "не нравиться", "не впечатлять" и т.д.). Зачем это нужно? Дело в том, что такие сочетания в подавляющем большинстве встречаются именно в отрицательных отзывах, тогда как просто лексемы "нравиться" и "впечатлять", скорее всего, есть в обоих типах отзывов, а значит, не попали в готовые тональные словари. А ведь это очень показательные моменты.
2. Во-вторых, сочетание "ничто/ничего + прилагательное" ("ничего интересного", "ничего оригинального" и т.д.). Опять же, такие сочетания характерны почти исключительно для плохих отзывов, а значит, могут значительно улучшить качество работы.
3. Наконец, сочетание "однозначно + прилагательное/глагол" (впечатление "однозначно положительное", "однозначно рекомендую" и т.д.). Ручной поиск показал, что такие сочетания очень характерны для положительных отзывов, а в отрицательных не встречаются (по крайней мере, в нашей выборке).

Считаем отзывы из файлов, созданных в первой домашке.

In [51]:
with open('good_reviews.txt', encoding='utf-8') as f:
    good_reviews = f.read().split('***')
with open('bad_reviews.txt', encoding='utf-8') as f:
    bad_reviews = f.read().split('***')

Возьмем из каждого списка столько, сколько плохих отзывов у нас имеется:

In [52]:
rev_num = len(bad_reviews)
good_reviews = '\n'.join(good_reviews[0:rev_num])
bad_reviews = '\n'.join(bad_reviews[0:rev_num])

Дальше сложная система pos-теггинга: Mystem работает гораздо, гораздо быстрее на целом тексте, чем на отдельных предложениях, но нам обязательно нужно сохранить деление на предложения, чтобы правильно выделять внутри них синтаксические группы. Поэтому поступим так:

1. Делим текст на предложения.
2. Каждое предложение очищаем от пунктуации и токенизируем.
3. Соединяем токены в предложения.
4. Соединяем предложения в текст, используя в качестве разделителя спецсимвол - знак доллара. Таким образом в дальнейшем мы сможем однозначно определить границу между предложениями.
5. Прогоняем весь текст через Mystem.
6. Проходимся по токенам: отбираем только словоформы и получаем их лемму и часть речи, пару "лемма-тег" записываем в список tagged_word, а tagged_word - в список one_sent. Если очередной токен - это знак доллара, то предложение закончилось: записываем one_sent в список parsed_sents, обнуляем one_sent и повторяем процесс. На выходе имеем список предложений, каждый из которых состоит из списков пар "лемма-тег". *(Да, это ужасно, я знаю.)*

In [53]:
def parse_reviews(text):
    sentences = sent_tokenize(text)
    punct = list(re.sub("[-']", '', string.punctuation))
    other_punct = ['``', '\"\"', '...', '--', '–', '—',
                   '«', '»', '“', '”', '’', '***', '…']
    all_punct = punct + other_punct
    rx = '[' + re.escape(''.join(all_punct)) + ']'
    tokenized_sents = []
    for sent in sentences:
        words = word_tokenize((re.sub(rx, ' ', sent)))
        tokenized_sents.append(' '.join(words))
    sent_text = '$'.join(tokenized_sents)

    parsed = m.analyze(sent_text)
    parsed_sents = []
    one_sent = []
    for token in parsed:
        tagged_word = []
        if 'analysis' in token or token['text'] == '$':
            if token['text'] == '$':
                parsed_sents.append(one_sent)
                one_sent = []
            else:
                if token['analysis']:
                    lemma = token['analysis'][0]['lex']
                    pos = token['analysis'][0]['gr'].split(
                        '=')[0].split(',')[0]
                    tagged_word.append(lemma)
                    tagged_word.append(pos)
                    one_sent.append(tagged_word)
    return parsed_sents

In [54]:
good_sents = parse_reviews(good_reviews)
bad_sents = parse_reviews(bad_reviews)

Я решила использовать код с семинара по SpaCy, т.к. мне кажется нелогичным использовать, например, Natasha Syntax Parser, если части речи я размечаю с помощью Mystem'а.

In [55]:
def get_groups(sents):
    not_verb = []
    nothing_adj = []
    def_adj_verb = []
    nothing = ['ничто', 'ничего']
    definitely = ['однозначно', 'однозначный']
    for sent in sents:
        length = len(sent)
        for i, token in enumerate(sent):
            if token[0] == 'не':
                if sent[i+1][1] == 'V':
                    group = token[0] + ' ' + sent[i+1][0]
                    not_verb.append(group)
            elif token[0] in nothing:
                if i + 1 < length:
                    if sent[i+1][1] == 'A':
                        group = token[0] + ' ' + sent[i+1][0]
                        nothing_adj.append(group)
            elif token[0] in definitely:
                if i + 1 < length:
                    if sent[i+1][1] == 'A' or sent[i+1][1] == 'V':
                        group = token[0] + ' ' + sent[i+1][0]
                        def_adj_verb.append(group)
    # для групп "не + глагол" установим порог частотности 2:
    not_verb = Counter(not_verb)
    not_verb = [k for k in list(not_verb.keys()) if not_verb[k] >= 2]
    all_groups = nothing_adj + def_adj_verb + not_verb
    return Counter(all_groups)

In [56]:
good_groups = get_groups(good_sents)
bad_groups = get_groups(bad_sents)

Остается создать множества уникальных биграмм:

In [57]:
good_bigrams = set(good_groups.keys())
bad_bigrams = set(bad_groups.keys())

In [58]:
common_bigrams = good_bigrams.intersection(bad_bigrams)
good_bigrams.difference_update(common_bigrams)
bad_bigrams.difference_update(common_bigrams)

In [59]:
good_bigrams

{'не бывать',
 'не доходить',
 'не думать',
 'не задумываться',
 'не казаться',
 'не лезть',
 'не оказываться',
 'не отпускать',
 'не отрываться',
 'не ошибаться',
 'не писать',
 'не пожалеть',
 'не показываться',
 'не покидать',
 'не поспорить',
 'не происходить',
 'не раздражать',
 'не разочаровываться',
 'не расстраивать',
 'не слышать',
 'не удаваться',
 'не хватать',
 'не ходить',
 'ничто конкретный',
 'ничто лишний',
 'ничто общий',
 'ничто плохой',
 'ничто сверхъестественный',
 'однозначно быть',
 'однозначно впечатлить',
 'однозначно заслуживать',
 'однозначно положительный',
 'однозначно рекомендовать',
 'однозначно согласный'}

In [60]:
bad_bigrams

{'не верить',
 'не вести',
 'не видеть',
 'не воспринимать',
 'не впечатлять',
 'не выдавать',
 'не делать',
 'не дорастать',
 'не ждать',
 'не зайти',
 'не зацеплять',
 'не идти',
 'не изменяться',
 'не иметься',
 'не купить',
 'не называть',
 'не находить',
 'не нести',
 'не нравиться',
 'не огорчать',
 'не одеваться',
 'не осиливать',
 'не открывать',
 'не отлипать',
 'не отличаться',
 'не переставать',
 'не повлечь',
 'не подходить',
 'не помнить',
 'не порекомендовать',
 'не портить',
 'не потерять',
 'не появляться',
 'не предполагать',
 'не придумывать',
 'не производить',
 'не проникаться',
 'не прочитывать',
 'не пугать',
 'не раскрывать',
 'не рекомендовать',
 'не связывать',
 'не смочься',
 'не собираться',
 'не советовать',
 'не соглашаться',
 'не спасать',
 'не способствовать',
 'не срастаться',
 'не тратить',
 'не трогать',
 'не уважать',
 'не увидеть',
 'не цеплять',
 'ничто великий',
 'ничто вкусный',
 'ничто годный',
 'ничто интересный',
 'ничто оригинальный',
 'ничто 

Теперь можно добавить получившиеся биграммы к тональным словарям из д/з 1 и для каждого отзыва проверять, есть ли в нем такие биграммы. Делать я этого не буду, потому что у меня горит другой дедлайн :(